In [2]:
!pip install pyngrok

In [ ]:
from fastapi import FastAPI
from pydantic import BaseModel, conlist
from typing import List
import numpy as np
import joblib
import tensorflow as tf

class Transaction(BaseModel):
    features: conlist(float, min_items=25, max_items=25)

app = FastAPI(
    title="Credit Card Fraud Detection API",
    description="Autoencoder + Ensemble + Calibration",
    version="1.0"
)

ae = None
encoder = None
scaler = None
stack_model = None
calibrator = None
threshold = None 

@app.on_event("startup")
def load_models():
    global ae, encoder, scaler, stack_model, calibrator, threshold
    ae = tf.keras.models.load_model("/content/ae_model.keras")
    encoder = tf.keras.models.load_model("/content/encoder_model.keras")
    scaler = joblib.load("/content/scaler.pkl")
    stack_model = joblib.load("/content/stack_model.pkl")
    calibrator = joblib.load("/content/calibrator.pkl")
    threshold = 0.037  

def extract_features(x_raw):
    x_scaled = scaler.transform([x_raw])
    x_reconstructed = ae.predict(x_scaled, batch_size=1)
    reconstruction_error = np.mean(np.abs(x_scaled - x_reconstructed), axis=1).reshape(-1,1)
    latent_features = encoder.predict(x_scaled, batch_size=1)
    final_features = np.hstack([latent_features, reconstruction_error])
    return final_features

@app.post("/predict_fraud")
async def predict(transaction: Transaction):
    try:
        input_data = np.array(transaction.features)
        features = extract_features(input_data)
        prob = stack_model.predict_proba(features)[:,1]
        calibrated = calibrator.predict_proba(prob.reshape(-1,1))[:,1]
        is_fraud = int(calibrated[0] >= threshold)
        return {
            "fraud_probability": float(calibrated[0]),
            "is_fraud": bool(is_fraud)
        }
    except Exception as e:
        return {"error": str(e)}


Overwriting main.py


In [ ]:
from pyngrok import ngrok
import nest_asyncio
import uvicorn

nest_asyncio.apply()
!ngrok config add-authtoken <your-token>
ngrok_tunnel = ngrok.connect(8000)
print('Public URL:', ngrok_tunnel.public_url)

uvicorn.run("main:app", host="0.0.0.0", port=8000)

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml
Public URL: https://a8b7-34-42-145-211.ngrok-free.app


INFO:     Started server process [167]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


INFO:     2409:40d6:1018:fd61:9862:171:b789:695a:0 - "GET /dpcs HTTP/1.1" 404 Not Found
INFO:     2409:40d6:1018:fd61:9862:171:b789:695a:0 - "GET /favicon.ico HTTP/1.1" 404 Not Found
INFO:     2409:40d6:1018:fd61:9862:171:b789:695a:0 - "GET /docs HTTP/1.1" 200 OK
INFO:     2409:40d6:1018:fd61:9862:171:b789:695a:0 - "GET /openapi.json HTTP/1.1" 200 OK
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 170ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 102ms/step
INFO:     2409:40d6:1018:fd61:9862:171:b789:695a:0 - "POST /predict HTTP/1.1" 200 OK


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
INFO:     2409:40d6:1018:fd61:9862:171:b789:695a:0 - "POST /predict HTTP/1.1" 200 OK


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [167]
